<a href="https://www.kaggle.com/code/tanvirsiddikemoin/purchase-analysis?scriptVersionId=115938439" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-purchase-history-from-electronics-store/kz.csv


In [2]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objs as go

from datetime import date

In [3]:
data = pd.read_csv('/kaggle/input/ecommerce-purchase-history-from-electronics-store/kz.csv')

data.head(5)

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18


In [4]:
data.dtypes

event_time        object
order_id           int64
product_id         int64
category_id      float64
category_code     object
brand             object
price            float64
user_id          float64
dtype: object

In [5]:
data.set_index('order_id', inplace=True)

In [6]:
null_columns = data.columns[data.isnull().any()]
data[null_columns].isnull().sum()

category_id       431954
category_code     612202
brand             506005
price             431954
user_id          2069352
dtype: int64

In [7]:
n_unique_products = data['product_id'].nunique()
n_unique_users = data['user_id'].nunique()
print('Number of unique users: ' + str(n_unique_users) +'. Number of unique products is: ' + str(n_unique_products))

Number of unique users: 98262. Number of unique products is: 25113


In [8]:
data['event_time']=pd.to_datetime(data['event_time'])

In [9]:
data.dropna(subset=['category_code'],inplace=True)

In [10]:
data['category'] = data['category_code'].str.rsplit('.', n=1, expand=True)[1]
data.drop(columns=['category_code'], inplace=True)

In [11]:
best_performing_brands = data.groupby('brand')['price'].sum().reset_index().sort_values('price', ascending=False).head(10)

In [12]:
fig = px.bar(
    best_performing_brands, 
    x='brand', 
    y='price', 
    title='Best performing brands',
    width=800, 
    height=800
)

fig.show()

In [13]:
most_sold_brands = data.groupby('brand')['price'].agg('count').reset_index().sort_values('price', ascending=False).head(10)
most_sold_brands.rename(columns={"brand": "brand", "price": "times_sold"}, inplace=True)

In [14]:
fig = px.bar(
    most_sold_brands, 
    x='brand', 
    y='times_sold', 
    title='Most sold brands',
    width=800, 
    height=800
)

fig.show()

In [15]:
most_sold_categories = data.groupby('category')['price'].agg('count').reset_index().sort_values('price', ascending=False).head(10)
most_sold_categories.rename(columns={"category": "category", "price": "times_sold"}, inplace=True)

In [16]:
fig = px.bar(
    most_sold_categories, 
    x='category', 
    y='times_sold', 
    title='Most sold categories',
    width=800, 
    height=800
)

fig.show()

In [17]:
best_time = data.groupby(data['event_time'].dt.strftime('%r'))['price'].sum().sort_values()[lambda x: x<= 900000].tail(10)

In [18]:
fig = px.bar(
    best_time,
    x="price", 
    orientation='h', 
    title="At what time most of the purchases were made", 
    width=800, 
    height=800
)

fig.show()

In [19]:
best_month = data.groupby(data['event_time'].dt.strftime('%B'))['price'].sum().sort_values()

In [20]:
fig = px.bar(
    best_month,
    x="price", 
    orientation='h', 
    title="In which months most of the purchases were made", 
    width=800, 
    height=800
)

fig.show()

In [21]:
# How much money spent 20% of top buyers in comparison with other 80% of clients
most_active_users = data.groupby('user_id')['price'].sum().reset_index().sort_values('price', ascending=False).head(round((data['user_id'].nunique())*0.2))
least_active_users = data.groupby('user_id')['price'].sum().reset_index().sort_values('price', ascending=False).tail(round((data['user_id'].nunique())*0.8))
top_20_percent_buyers = most_active_users['price'].sum()
bottom_80_percent_buyers = least_active_users['price'].sum()
last_data = pd.DataFrame(data={'most_active': [most_active_users['price'].sum()], 'least_active': [least_active_users['price'].sum()]})

In [22]:
fig = px.bar(
    last_data,  
    title="20% of most active in comparison with other 80% of buyers", 
    width=800, 
    height=800
)

fig.show()